In [ ]:
cd /kaggle/input/playground-series-s3e6/sample_submission.csv

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv("/kaggle/input/playground-series-s3e6/train.csv")

In [ ]:
train

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train = train.astype({"id":"category","hasYard":"category","hasPool":"category","cityPartRange":"category","isNewBuilt":"category","hasStormProtector":"category","hasStorageRoom":"category"})

In [ ]:
train.dtypes

In [ ]:
train.describe().T

In [ ]:
(train.isnull().sum()/len(train))*100

In [ ]:
train.nunique()

In [ ]:
train = train.drop(["id","cityCode"],axis =1)

In [ ]:
train

In [ ]:
train.duplicated().sum()

In [ ]:
train.columns

In [ ]:
num_cols = ["squareMeters","numberOfRooms","floors","numPrevOwners","made","basement","attic","garage","hasGuestRoom"]
cat_cols = ["hasYard","hasPool","cityPartRange","isNewBuilt","hasStormProtector","hasStorageRoom"]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#boxplot for all the numerical attributes in dataframe
train.plot(kind ='box',subplots = True,sharex= False,sharey=False,figsize=(15,15))
plt.show()

In [ ]:
# detecting ouliers for all before removing 
Q1 = train[num_cols].quantile(0.25)
Q3 = train[num_cols].quantile(0.75)
IQR = Q3-Q1

before_rvm_detect_outliers = ((train[num_cols]<(Q1- 1.5*IQR)) | (train[num_cols]>(Q3+ 1.5*IQR))).sum()
before_rvm_detect_outliers

In [ ]:
detected_outlier_cols = ["squareMeters","floors","made","basement","attic","garage"]

Q1 = train[detected_outlier_cols].quantile(0.25)
Q3 = train[detected_outlier_cols].quantile(0.75)
IQR = Q3-Q1

train = train[~((train[detected_outlier_cols] < (Q1 - 1.5 * IQR)) |(train[detected_outlier_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
train.shape

In [ ]:
# detecting ouliers for all before removing 
after_rvm_detect_outliers = ((train[num_cols]<(Q1- 1.5*IQR)) | (train[num_cols]>(Q3+ 1.5*IQR))).sum()
after_rvm_detect_outliers

In [ ]:
#boxplot for all the numerical attributes in dataframe
train.plot(kind ='box',subplots = True,sharex= False,sharey=False,figsize=(15,15))
plt.show()

In [ ]:
X = train.drop(["price"],axis = 1)
y = train["price"]

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3,random_state=107)

In [ ]:
print("X_train is:", X_train.shape)
print("X_val is:", X_val.shape)
print("X_train is:", y_train.shape)
print("X_val is:", y_val.shape)

In [ ]:
X_train

In [ ]:
X_val

In [ ]:
y_train

In [ ]:
y_val

In [ ]:
print(X_train.isnull().sum().sum())
print(X_val.isnull().sum().sum())
print(y_train.isnull().sum().sum())
print(y_val.isnull().sum().sum())

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# fitting the X_train[num_cols] in standardization model
scaler = StandardScaler()
scaler = scaler.fit(X_train[num_cols])

In [ ]:
# fitting the X_train[num_cols] and X_test[num_cols] in standardization model
X_train_std = pd.DataFrame(scaler.transform(X_train[num_cols]), columns = X_train[num_cols].columns)
X_val_std = pd.DataFrame(scaler.transform(X_val[num_cols]), columns = X_val[num_cols].columns)

In [ ]:
X_train_std

In [ ]:
X_val_std

In [ ]:
# resetting the index to concat correctly
X_train_std = X_train_std.reset_index()
X_train_std

In [ ]:
# dropping the index column 
X_train_std = X_train_std.drop(["index"], axis = 1)
X_train_std

In [ ]:
# resetting the index to concat correctly
X_val_std = X_val_std.reset_index()
X_val_std

In [ ]:
# dropping the index column 
X_val_std = X_val_std.drop(["index"], axis = 1)
X_val_std

In [ ]:
# creationg dummies for cat_cols from X_train
X_train_dummies = pd.get_dummies(X_train[cat_cols],drop_first = True)

In [ ]:
X_train_dummies

In [ ]:
# creationg dummies for cat_cols from X_test
X_val_dummies = pd.get_dummies(X_val[cat_cols],drop_first = True)

In [ ]:
X_val_dummies

In [ ]:
X_train_dummies = X_train_dummies.reset_index()
X_train_dummies

In [ ]:
X_train_dummies = X_train_dummies.drop(["index"],axis = 1)
X_train_dummies

In [ ]:
X_val_dummies = X_val_dummies.reset_index()
X_val_dummies

In [ ]:
X_val_dummies = X_val_dummies.drop(["index"],axis = 1)
X_val_dummies

In [ ]:
final_X_train = pd.concat([X_train_std,X_train_dummies],axis = 1)
final_X_train

In [ ]:
final_X_val = pd.concat([X_val_std,X_val_dummies],axis = 1)
final_X_val

In [ ]:
final_X_train.isnull().sum()

In [ ]:
final_X_val.isnull().sum()

# Model Building

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid={'n_jobs': [1,5,9,10,25,45,76,100,-1]
            }

clf=LinearRegression()
grid_search=GridSearchCV(clf,param_grid)
grid_search.fit(final_X_train,y_train)

In [ ]:
print(grid_search.best_params_)

In [ ]:
lr = LinearRegression(n_jobs=1)
lr = lr.fit(final_X_train,y_train)
lr

In [ ]:
lr_y_train_predicts = lr.predict(final_X_train)
lr_y_train_predicts

In [ ]:
lr_y_val_predicts = lr.predict(final_X_val)
lr_y_val_predicts

In [ ]:
# To represent the results in a data frame:
from sklearn import metrics
import numpy as np
scores = pd.DataFrame(columns=['Model','MAE_Train','MSE_Train','RMSE_Train','MAPE_Test','MAE_Test','MSE_Test','RMSE_Test','MAPE_Test'])

def get_metrics(train_act,train_pred,test_act,test_pred,model_description,dataframe):
    MAE_Train = metrics.mean_absolute_error(train_act,train_pred)
    MSE_Train = metrics.mean_squared_error(train_act,train_pred)
    RMSE_Train = np.sqrt(metrics.mean_squared_error(train_act,train_pred))
    MAPE_Train = metrics.mean_absolute_percentage_error(train_act,train_pred)
    MAE_Test = metrics.mean_absolute_error(test_act,test_pred)
    MSE_Test = metrics.mean_squared_error(test_act,test_pred)
    RMSE_Test = np.sqrt(metrics.mean_squared_error(test_act,test_pred))
    MAPE_Test = metrics.mean_absolute_percentage_error(test_act,test_pred)
       
    dataframe=dataframe.append(pd.Series([model_description,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test],index=scores.columns ), ignore_index=True)
    
    return(dataframe)

In [ ]:
errors=get_metrics(y_train,lr_y_train_predicts,y_val,lr_y_val_predicts,'Linear_model',scores)
errors